# User-Based Recommender System

A recommender system is an algorithm or model that takes in information about a user and suggests an item — new to them — that is likely to be of interest. There are several approaches to building such a system, and this notebook will focus on **user-based methods**. Let's begin by starting the PySpark session:


In [1]:
# Load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import helper_functions as hf
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType, StringType, ArrayType
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline

# Initialize Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CBRS").getOrCreate()
spark

In [2]:
# Movies Metadata (Load Dataset)
df = spark.read.parquet('data/cleaned/ratings/')
display(df.limit(3).toPandas())

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0


## Introduction


User-Based Recommender Systems (UBRS) use the information about the preferences of similar users — or opposites — to make recommendations. The core idea is that if two users show high correlation in their preferences — positive or negative —, then one user's future preferences can be predicted based on the other user's past behavior. However, preferences not only have to refer to product ratings, but they include for example interactions with the webpage — time spent reading an article, clicked items in the current session, photos watched, etc. For this reason, an UBRS is capable of providing a custom experience even session wise, although the reliability of the data becomes the main issue.

On the contrary, this kind of systems struggles with several situations:
- **Cold Start Problem:** when there is little to no information about a customer, the recommendations become unreliable, as the user cannot show correlation with any other.
- **Computation:** when the database is large, the process becomes expensive.
- **Changes of opinion:** while the item data is theoretically unchanging, the customer's opinion about the rated items may vary over time.

### Cosine Similarity

Through cosine similarity, it is possible to compute a value based on the *angle* between two *vectors*, with the *vectors* being the user's rating of movies and the *angle* the similarity to each other — it can be positive or negative, if they are similar or opposites, but the values are kept between $[-1,1]$.

The main problem with this approach are the memory issues, caused by the large number of unique movies and users in the dataset, but it is still interesting to understand this implementation and, for that reason, we will continue with a limited data set of 10 users:

In [3]:
# Filter out users with an Id higher than 10
filtered_df = df.filter(f.col('userId') <= 10)

# Build the user-movie matrix: 
#   userId as rows, movieId as columns and user rating as content.
user_movie = filtered_df.groupBy('userId').pivot('movieId').agg(f.first('rating'))

With the memory constrains solved, the next problem is that there are users who did not rate every movie we are considering, which is the most common case, possibly because they have not watched it. This means that the user-movie matrix is filled with lots of missing data (`NA`), which cannot be easily imputed by a number, as it would assume it to be the opinion of the user about the movie.

There are several approaches to manage this:
- Impute the missing values as $0$, which is simple, fast and allows for computing cosine similarity, at the expense of some reliability loss.
- Only compare overlapping results, meaning that the similarity will be computed based on the movies that both users rated, and if they do not share any, the similarity score is zero. This is similar to how Pearson correlation is usually computed.
- Mean centering, although for users with only one rating, it would cause them to have $0$ in all their rating.

(??)

In [ ]:
# Impute NAs as 0
user_movie = user_movie.fillna(0)

# Create a dense vector representation for each user
vector_assembler = VectorAssembler(
    inputCols=user_movie.columns[1:], outputCol='vectors'
)

# Normalization
scaler = MinMaxScaler(inputCol='vectors', outputCol='features')

# Pipeline
p = Pipeline(stages=[vector_assembler, scaler]).fit(user_movie)

# Transform data
user_vector = p.transform(user_movie).select('userId','features')
user_vector.show(3, truncate=True)

+------+--------------------+
|userId|            features|
+------+--------------------+
|     1|(352,[11,13,60,79...|
|     6|(352,[3,73,149,15...|
|     3|(352,[34,36,37,60...|
+------+--------------------+
only showing top 3 rows



In [9]:
# Cross Join: cartesian product to form pairs (filter to avoid redundancy)
user_cross = user_vector.alias('left').\
    crossJoin(user_vector.alias('right')).\
    filter(f.col('left.userId') < f.col('right.userId'))

display(user_cross.limit(3).toPandas())

,userId,features,userId,features
0,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
# Cosine Similarity UDF
def cosine_similarity(v1,v2):
    # Formula:
    #   Sim = A·B / |A||B|

    # Numerator: scalar product
    num = sum(c1*c2 for (c1,c2) in zip(v1,v2))
    
    # Denominator: modules
    mod_a = np.sqrt(sum(c1**2 for c1 in v1))
    mod_b = np.sqrt(sum(c2**2 for c2 in v2))
    den = mod_a * mod_b

    # Similarity
    return float(num) / float(den) if den != 0.0 else 0.0

cosine_udf = f.udf(lambda v1,v2: cosine_similarity(v1,v2), FloatType())

# Apply udf to the pairs of movies in every row
df_similarities = user_cross.\
    withColumn('cosine', cosine_udf(f.col('left.features'), f.col('right.features'))).\
    select('left.userId', 'right.userId', 'cosine')

display(df_similarities.limit(10).toPandas())

,userId,userId,cosine
0,1,6,0.000000
1,1,3,0.109584
2,1,5,0.039579
3,1,9,0.076114
4,1,4,0.068095
5,1,8,0.021307
6,1,7,0.159225
7,1,10,0.000000
8,1,2,0.000000
9,6,9,0.000000
